In [ ]:
#Import required modules
import requests
import pandas as pd
import io
import zipfile
from datetime import datetime, timedelta
import time
import random
import numpy as np

Please note there is no need to run this code as the data file has already been included in the zip file. Since the formatting for options data for the year 2025 was not conistent on bhavcopy for infosys, due to the change in format on july 2024 to another one called Udiff, we were getting inconsistent data to work with, so for more accuracy we used data from July 2022 to July 2024.

In [ ]:
SYMBOL = "INFY"
OUTPUT_FILE = "infy_atm_current_month_expiry.csv"

# Time line for which we are collecting data - (July 2022 to July 2024) 
START_DATE = datetime(2022, 7, 1)
END_DATE = datetime(2024, 7, 31)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://www.nseindia.com/"
}

def get_bhavcopy_url(date_obj, type="fo"):
    d_str = date_obj.strftime("%d%b%Y").upper()
    year = date_obj.strftime("%Y")
    month = date_obj.strftime("%b").upper()
    filename = f"{type}{d_str}bhav.csv.zip"
    
    base = "https://nsearchives.nseindia.com/content/historical"
    if type == "cm":
        return f"{base}/EQUITIES/{year}/{month}/{filename}"
    else:
        return f"{base}/DERIVATIVES/{year}/{month}/{filename}"

def download_data(session, url):
    try:
        response = session.get(url, headers=HEADERS, timeout=10)
    except Exception:
        pass
    return None

def get_nearest_strike(spot_price, available_strikes):
    available_strikes = available_strikes.astype(float)
    idx = (np.abs(available_strikes - spot_price)).argmin()
    return available_strikes.iloc[idx]

In [ ]:
if __name__ == "__main__":
    date_list = []
    curr = START_DATE
    while curr <= END_DATE:
        if curr.weekday() < 5: 
            date_list.append(curr)
        curr += timedelta(days=1)

    print(f"Target: {len(date_list)} days | Mode: STEALTH")
    print("Logic: Current Month Expiry only.")
    print("-" * 80)
    
    session = requests.Session()
    results = []
    
    for i, current_date in enumerate(date_list):
        date_str = current_date.strftime("%Y-%m-%d")
        print(f"[{i+1}/{len(date_list)}] {date_str}...", end=" ", flush=True)
        
        time.sleep(random.uniform(1.0, 3.0)) # Jitter

        cm_df = download_data(session, get_bhavcopy_url(current_date, "cm"))
        
        if isinstance(cm_df, str) and cm_df == "BLOCKED":
            print("! BLOCKED. Change IP.")
            break
        if cm_df is None:
            print("No Data")
            continue
            
        try:
            spot_row = cm_df[cm_df['SYMBOL'] == SYMBOL]
            if spot_row.empty:
                print("Symbol Not Found")
                continue
            spot_price = float(spot_row['CLOSE'].values[0])
            fo_df = download_data(session, get_bhavcopy_url(current_date, "fo"))
            
            if isinstance(fo_df, str) and fo_df == "BLOCKED":
                print("! BLOCKED. Change IP.")
                break
            if fo_df is None:
                print("FO Missing")
                continue

            fo_df.columns = fo_df.columns.str.strip()
            infy_opts = fo_df[(fo_df['SYMBOL'] == SYMBOL) & (fo_df['INSTRUMENT'] == 'OPTSTK')].copy()
            
            infy_opts['EXPIRY_DT'] = pd.to_datetime(infy_opts['EXPIRY_DT'], format='%d-%b-%Y')
            valid_expiries = infy_opts[infy_opts['EXPIRY_DT'] >= current_date]
            
            if valid_expiries.empty:
                print("No Valid Expiry")
                continue
            
            # Current Month Expiry
            nearest_expiry = valid_expiries['EXPIRY_DT'].min()
            
            # Filter Data to nearest strike price
            curr_opts = infy_opts[infy_opts['EXPIRY_DT'] == nearest_expiry]
            atm_strike = get_nearest_strike(spot_price, pd.Series(curr_opts['STRIKE_PR'].unique()))
            
            atm_rows = curr_opts[curr_opts['STRIKE_PR'] == atm_strike]
            ce = atm_rows[atm_rows['OPTION_TYP'] == 'CE']
            pe = atm_rows[atm_rows['OPTION_TYP'] == 'PE']
            
            ce_price = ce['CLOSE'].values[0] if not ce.empty else 0
            pe_price = pe['CLOSE'].values[0] if not pe.empty else 0
            
            results.append({
                "Date": date_str,
                "Spot": spot_price,
                "Expiry": nearest_expiry.strftime("%Y-%m-%d"), # Moved up
                "Strike": atm_strike,
                "Call": ce_price,
                "Put": pe_price
            })
            print(f"OK | Exp: {nearest_expiry.strftime('%Y-%m-%d')}")

        except Exception as e:
            print(f"Err: {e}")

    if results:
        pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)
        print(f"\nDone. Saved to {OUTPUT_FILE}")

Target: 544 days | Mode: STEALTH
Logic: Current Month Expiry only.
--------------------------------------------------------------------------------
[1/544] 2022-07-01... 

OK | Exp: 2022-07-28
[2/544] 2022-07-04... OK | Exp: 2022-07-28
[3/544] 2022-07-05... OK | Exp: 2022-07-28
[4/544] 2022-07-06... OK | Exp: 2022-07-28
[5/544] 2022-07-07... OK | Exp: 2022-07-28
[6/544] 2022-07-08... OK | Exp: 2022-07-28
[7/544] 2022-07-11... OK | Exp: 2022-07-28
[8/544] 2022-07-12... OK | Exp: 2022-07-28
[9/544] 2022-07-13... OK | Exp: 2022-07-28
[10/544] 2022-07-14... OK | Exp: 2022-07-28
[11/544] 2022-07-15... OK | Exp: 2022-07-28
[12/544] 2022-07-18... OK | Exp: 2022-07-28
[13/544] 2022-07-19... OK | Exp: 2022-07-28
[14/544] 2022-07-20... OK | Exp: 2022-07-28
[15/544] 2022-07-21... OK | Exp: 2022-07-28
[16/544] 2022-07-22... OK | Exp: 2022-07-28
[17/544] 2022-07-25... OK | Exp: 2022-07-28
[18/544] 2022-07-26... OK | Exp: 2022-07-28
[19/544] 2022-07-27... OK | Exp: 2022-07-28
[20/544] 2022-07-28... OK | Exp: 2022-07-28
[21/544] 2022-07-29... OK | Exp: 2022-08-25
[22/544] 2022-08-01... OK | Exp: 2022-08-25
[23/544] 2022-08-02... OK | Exp: 2022-08-25
[24/544] 2022-08-03